In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, GCNConv
from scipy.sparse import coo_matrix
import numpy as np
from torch_geometric.utils import to_networkx
import random
from heapdict import heapdict
from node2vec import Node2Vec
import argparse
import torch.nn.init as init
from utils import *
from numpy.linalg import norm

In [2]:
# 定义GCN模型
class GCN(torch.nn.Module):
    def __init__(self,num_features, graph_embedding_size):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 128)
        self.conv2 = GCNConv(128, graph_embedding_size)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# 定义GAT模型
class GAT(torch.nn.Module):
    def __init__(self, num_features, num_heads=4, graph_embedding_size=256):
        super(GAT, self).__init__()
        self.gat1 = GATConv(num_features, 512, heads=num_heads, dropout=0.2)
        self.gat2 = GATConv(512 * num_heads, graph_embedding_size, heads=1, concat=False, dropout=0.2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.gat2(x, edge_index)
        return x

# 定義用來決定edge是否修改的MLP
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 2)
        self.fc5 = nn.Linear(128, 1)

        init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc4.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc5.weight, nonlinearity='relu')
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        pro = torch.softmax(self.fc4(x), dim=1)
        binary_decision = torch.sigmoid(self.fc5(x))
        return pro, binary_decision
    
class GCN_edge_modify(nn.Module):
    def __init__(self, num_features, hidden_channels = 512):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        # 最后一层，用于产生最终输出
        self.out = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.out(x)
        return torch.sigmoid(x)
    
class MLPClassifier(nn.Module):  #最後用來判定graph的result是否有相同的MVC
    def __init__(self, input_size):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)  # 第一层
        self.fc2 = nn.Linear(512, 256)          # 第二层
        self.fc3 = nn.Linear(256, 128)          # 第三层
        self.fc4 = nn.Linear(128, 1)           # 输出层

        init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc4.weight, nonlinearity='relu')
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))        # 使用sigmoid确保输出在0到1之间
        return x
    
class MVC_Predict(nn.Module):  #最後用來判定graph的result是否有相同的MVC
    def __init__(self, input_size):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)  # 第一层
        self.fc2 = nn.Linear(512, 256)          # 第二层
        self.fc3 = nn.Linear(256, 128)          # 第三层
        self.fc4 = nn.Linear(128, 1)           # 输出层

        init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc4.weight, nonlinearity='relu')
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)       # 使用sigmoid确保输出在0到1之间
        return x

In [3]:
class Modify_edge(nn.Module):
    def __init__(self, num_features, graph_embedding_size, epoch = 100, lr = 0.0001, modified_edge = 30, GraphNumber = 50, Graphsize = 50, num_heads = 4):
        super(Modify_edge, self).__init__()
        self.connect_info_num = 5
        self.gat = GAT(num_features=num_features + 1, num_heads=num_heads, graph_embedding_size = graph_embedding_size)  # 根据需要调整头数
        self.mlp = MLP(input_size=3 * graph_embedding_size + self.connect_info_num)
        self.classifier = MLPClassifier(input_size=2 * graph_embedding_size)
        self.modified_edge = modified_edge
        self.lr = lr
        self.GraphNumber = GraphNumber
        self.Graphsize = Graphsize
        self.Node2Vec_feature = num_features
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.best_preserve = 0
        
        
    def forward(self):
        self.modified_graphs = []
        self.edge_dict = {}
        self.whole_edge_set = set()
        self.dataset = []
        self.init_graph()  ## create self.dataset
        
        original_embeddings = []
        for data in self.dataset:
            data = data.to(self.device)
            embedding = self.gat(data)
            original_embeddings.append(embedding)
        self.original_graph_embeddings = self.get_graph_embedding(original_embeddings)
        self.mlp_label = []
        self.mlp_target = []
        for emb, data, graph_emb in zip(original_embeddings, self.dataset, self.original_graph_embeddings):
            modify_edge, edge_label, edge_softmax = self.generate_edge_embeddings(data, emb, graph_emb)
            self.mlp_label.append(edge_label)
            self.mlp_target.append(edge_softmax)
            # print(f"mlp_label grad : {edge_label.requires_grad}, mlp_target grad : {edge_softmax.requires_grad}")
            add_num = 0
            delete_num = 0
            G = to_networkx(data, to_undirected=True)
            G_edge_set = set(G.edges())
            for edge in modify_edge:  # decision: (probabilities,(u,v))
                if (G.has_edge(edge[0], edge[1])):
                    G.remove_edge(edge[0], edge[1])
                    delete_num += 1
                else:
                    G.add_edge(edge[0], edge[1])
                    add_num += 1
            self.modified_graphs.append(G)
        
        self.modified_dataset = []  #type pyg
        for G in self.modified_graphs:
            # 从 NetworkX 图创建边索引
            edge_index = torch.tensor(list(G.edges)).t().contiguous()
            vec = Node2Vec(G, dimensions=self.Node2Vec_feature, walk_length=10, num_walks=10, workers=4, quiet=True)
            InitNodeEmb = vec.fit(window=5, min_count=1, batch_words=4)
            # embeddings = InitNodeEmb.wv # 50 node * 50 features
            node2vec_embeddings = [InitNodeEmb.wv[str(i)] for i in range(len(G.nodes))]
            node2vec_features_np = np.array(node2vec_embeddings)
            degrees = nx.degree(G)
            degree_values = [deg for node, deg in degrees]
            degree_values = torch.tensor(degree_values, dtype=torch.float)
            degree_normalized = (degree_values - degree_values.min()) / (degree_values.max() - degree_values.min())
            degree_features = degree_normalized.view(-1, 1)
            node2vec_features = torch.tensor(node2vec_features_np)
            combined_features = torch.cat([degree_features, node2vec_features], dim=1)
            # 使用单位矩阵作为节点特征
            # x = torch.eye(G.number_of_nodes())
            
            # 创建 Data 对象
            data = Data(x=combined_features, edge_index=edge_index)
            self.modified_dataset.append(data)  
            
        modified_embeddings = []
        for data in self.modified_dataset:
            data = data.to(self.device)
            embedding = self.gat(data)
            modified_embeddings.append(embedding)  
        self.modified_graph_embeddings = self.get_graph_embedding(modified_embeddings)
        cos = nn.CosineSimilarity(dim=1)
        self.cosine_similarities = cos(self.modified_graph_embeddings, self.original_graph_embeddings).mean()
            
        labels = []
        MVC_diff = 0
        for mod_graph, orig_mvc in zip(self.modified_graphs, self.train_opt):
            mod_mvc = len(self.calculate_MVC(mod_graph))
            print(f"mod_mvc: {mod_mvc}, ori_mvc: {orig_mvc}, diff: {abs(mod_mvc - orig_mvc)}")
            MVC_diff = MVC_diff + abs(mod_mvc - orig_mvc) * abs(mod_mvc - orig_mvc)
            label = 1 if mod_mvc == orig_mvc else 0
            labels.append(label)
        print(f"label presreved: {labels.count(1)}, MVC_diff : {MVC_diff}")
        label_preserve = labels.count(1)
        if label_preserve > self.best_preserve:
            self.best_preserve = label_preserve
            torch.save(self.state_dict(), "/workspace/Model/Modify_edge_model_1.pth")
        combined_embeddings = [torch.cat((mod_emb, orig_emb)) for mod_emb, orig_emb in zip(self.modified_graph_embeddings, self.original_graph_embeddings)]
        # 将嵌入和标签转换为张量
        combined_embeddings_tensor = torch.stack(combined_embeddings)
        # combined_embeddings_tensor shape : torch.Size([50, 2*graph embedding]) 兩張graph的嵌入拼接起來
        self.labels_tensor = torch.tensor(labels).to(self.device)
        # labels_tensor shape : torch.Size([50]) 也就是50個graph的label
        self.preserve_predict = self.classifier(combined_embeddings_tensor).squeeze()
        # preserve_predict shape: torch.Size([50])也就是50個graph預測的label
        
        self.mlp_label = torch.stack(self.mlp_label, dim=0) # shape: torch.Size([50, 1225])
        self.mlp_target = torch.stack(self.mlp_target, dim=0) # shape: torch.Size([50, 1225])

        
        return self.cosine_similarities, self.preserve_predict, self.labels_tensor, MVC_diff/self.GraphNumber, label_preserve, self.mlp_label, self.mlp_target
        
        
    def init_graph(self):
        """construct or load training graph and use Node2vec to get node embedding"""
        self.train_graphs, self.train_opt = pickle_load("/workspace/Synthetic_graph/Training_graph_50_withOPT.pkl")
        self.create_edge_dict(self.Graphsize)
        for i in range(self.GraphNumber):
            # p = random.uniform(graph_density[0], graph_density[1])
            # G = nx.erdos_renyi_graph(graph_size, p)
            G = self.train_graphs[i]
            adj_matrix = nx.adjacency_matrix(G)
            adj_matrix = coo_matrix(adj_matrix)

            row = torch.from_numpy(adj_matrix.row.astype(np.int64))
            col = torch.from_numpy(adj_matrix.col.astype(np.int64))
            edge_index = torch.stack([row, col], dim=0)
            vec = Node2Vec(G, dimensions=self.Node2Vec_feature, walk_length=10, num_walks=10, workers=4, quiet=True)
            InitNodeEmb = vec.fit(window=5, min_count=1, batch_words=4)
            # embeddings = InitNodeEmb.wv # 50 node * 50 features
            node2vec_embeddings = [InitNodeEmb.wv[str(i)] for i in range(len(G.nodes))]
            node2vec_features_np = np.array(node2vec_embeddings)
            degrees = nx.degree(G)
            degree_values = [deg for node, deg in degrees]
            degree_values = torch.tensor(degree_values, dtype=torch.float)
            degree_normalized = (degree_values - degree_values.min()) / (degree_values.max() - degree_values.min())
            degree_features = degree_normalized.view(-1, 1)
            node2vec_features = torch.tensor(node2vec_features_np)
            combined_features = torch.cat([degree_features, node2vec_features], dim=1)
            
            # x = torch.tensor(embeddings.vectors, dtype=torch.float32)
            # x = torch.eye(G.number_of_nodes())  # 节点特征

            data = Data(x=combined_features, edge_index=edge_index)
            self.dataset.append(data)
            
    def create_edge_dict(self,graph_size):
        """mapping edge to index"""
        index = 0
        for i in range(graph_size - 1):
            for j in range(i + 1, graph_size):
                self.whole_edge_set.add((i, j))
                self.edge_dict[(i, j)] = index
                index += 1
        
    def calculate_MVC(self,graph, UB=9999999, C=set()):
        """use branch and bound to find out the mvc result"""
        if len(graph.edges()) == 0:
            return C

        v, _ = max(graph.degree(), key=lambda a: a[1])
        # C1 分支：選擇鄰居
        C1 = C.copy()
        neighbors = set(graph.neighbors(v))
        C1.update(neighbors)
        graph_1 = graph.copy()
        graph_1.remove_nodes_from(neighbors)
        if len(C1) < UB:
            C1 = self.calculate_MVC(graph_1, UB, C1)

        # C2 分支：只選擇該節點
        C2 = C.copy()
        C2.add(v)
        graph_2 = graph.copy()
        graph_2.remove_node(v)
        if len(C2) < UB:
            C2 = self.calculate_MVC(graph_2, min(UB, len(C1)), C2)

        return min(C1, C2, key=len)
    
    def predict_MVC (self,graph_emb):
        """predict the result of MVC"""
        for emb in graph_emb:
            result = self.MVC_predict(emb)
            print(result)
    
    def get_graph_embedding(self,embeddings):
        """average all node embeddings to get graph embedding, embedding 是一個list，每個元素是一個graph的node embedding"""
        graph_embeddings = []
        for embedding in embeddings:
            graph_embedding = embedding.mean(dim=0)  # 对所有节点嵌入求平均
            graph_embeddings.append(graph_embedding)
        return torch.stack(graph_embeddings)
    
    def generate_edge_embeddings(self,data, embedding, graph_emb):
        """generate and sample edge embeddings for training 需要修改"""
        data= to_networkx(data, to_undirected=True)
        edge_set = set(data.edges()) 
        edge_modify_label = [torch.tensor(0.0) for _ in range(len(self.whole_edge_set))]
        edge_modify_softmax = []
        edge_pro_list = []
        none_edge_pro_list = []
        gumbel_modify_edge = []  #紀錄那些邊被修改
        for u,v in self.whole_edge_set:
            node1_emb = embedding[u]
            node2_emb = embedding[v]
            if (u,v) in edge_set:
                connect_info = torch.tensor([1.0]*self.connect_info_num).to(self.device)
                node_pair_emb = torch.cat([node1_emb, node2_emb,connect_info , graph_emb])
                node_pair_emb = node_pair_emb.unsqueeze(0)
                probabilities, decision = self.mlp(node_pair_emb)
                # print(f"shape of probabilities: {probabilities.shape}, type: {type(probabilities)}, probabilities: {probabilities}, decision: {decision}, shape of decision: {decision.shape}")
                modify_pro = F.gumbel_softmax(probabilities, tau=1, hard=True)[0][1] #直接是0或1
                edge_pro_list.append((decision,(u,v)))
                edge_modify_softmax.append(modify_pro)
                gumbel_modify_edge.append((u,v))

            else:
                connect_info = torch.tensor([0.0]*self.connect_info_num).to(self.device)
                node_pair_emb = torch.cat([node1_emb, node2_emb, connect_info, graph_emb])
                node_pair_emb = node_pair_emb.unsqueeze(0)
                probabilities, decision = self.mlp(node_pair_emb)
                # print(f"shape of probabilities: {probabilities.shape}, type: {type(probabilities)}, probabilities: {probabilities}, decision: {decision}, shape of decision: {decision.shape}")
                modify_pro = F.gumbel_softmax(probabilities, tau=1, hard=True)[0][1] #直接是0或1
                none_edge_pro_list.append((decision,(u,v)))
                edge_modify_softmax.append(modify_pro)
                gumbel_modify_edge.append((u,v))
        
        """計算label不需用gradient"""
        edge_pro_list = sorted(edge_pro_list, key=lambda x: x[0], reverse=True)
        none_edge_pro_list = sorted(none_edge_pro_list, key=lambda x: x[0], reverse=True)
        for i in range(len(edge_pro_list)):
            if i < self.modified_edge:
                index = self.edge_dict[edge_pro_list[i][1]]
                edge_modify_label[index] = torch.tensor(1.0)
        for i in range(len(none_edge_pro_list)):
            if i < self.modified_edge:
                index = self.edge_dict[none_edge_pro_list[i][1]]
                edge_modify_label[index] = torch.tensor(1.0)
        # print(f"len of edge_modify_label : {len of edge_modify_softmax : {len(edge_modify_softmax)}, len(edge_modify_label)}, type of edge_modify_softmax[0] : {type(edge_modify_softmax[0])}, type of edge_modify_label[0] : {type(edge_modify_label[0])}")
        # print(f"true in edge_modify_softmax : {edge_modify_softmax.count(torch.tensor(1.0))}, true in edge_modify_label : {edge_modify_label.count(torch.tensor(1.0))}, edge_modify_label : { edge_modify_label}")
        
        combined_embeddings = gumbel_modify_edge
        edge_modify_label = torch.stack(edge_modify_label, dim=0)
        edge_modify_softmax = torch.stack(edge_modify_softmax, dim=0)
        
        return combined_embeddings, edge_modify_label, edge_modify_softmax
    
    def validation(self):
        validation_data, valid_opt, pro = pickle_load("/workspace/Synthetic_graph/Validation_graph_200_withOPTPRO.pkl")
        valid_original_embeddings = []
        self.valid_dataset = []
        self.valid_modified_graphs = []
        # 把validation data轉成pyg的data，並透過node2vec得到node feature，再輸入GAT得到node embedding，最後透過get_graph_embedding得到graph embedding
        for i in range(len(validation_data)):
            G = validation_data[i]
            adj_matrix = nx.adjacency_matrix(G)
            adj_matrix = coo_matrix(adj_matrix)

            row = torch.from_numpy(adj_matrix.row.astype(np.int64))
            col = torch.from_numpy(adj_matrix.col.astype(np.int64))
            edge_index = torch.stack([row, col], dim=0)
            vec = Node2Vec(G, dimensions=self.Node2Vec_feature, walk_length=10, num_walks=10, workers=4, quiet=True)
            InitNodeEmb = vec.fit(window=5, min_count=1, batch_words=4)
            # embeddings = InitNodeEmb.wv # 50 node * 50 features
            node2vec_embeddings = [InitNodeEmb.wv[str(i)] for i in range(len(G.nodes))]
            node2vec_features_np = np.array(node2vec_embeddings)
            degrees = nx.degree(G)
            degree_values = [deg for node, deg in degrees]
            degree_values = torch.tensor(degree_values, dtype=torch.float)
            degree_normalized = (degree_values - degree_values.min()) / (degree_values.max() - degree_values.min())
            degree_features = degree_normalized.view(-1, 1)
            node2vec_features = torch.tensor(node2vec_features_np)
            combined_features = torch.cat([degree_features, node2vec_features], dim=1)
            
            # x = torch.tensor(embeddings.vectors, dtype=torch.float32)

            data = Data(x=combined_features, edge_index=edge_index)
            self.valid_dataset.append(data)
            data = data.to(self.device)
            embedding = self.gat(data)
            valid_original_embeddings.append(embedding)
        self.valid_graph_embedding = self.get_graph_embedding(valid_original_embeddings)
        
        # 將每個graph的node embedding和graph embedding還有pyg data輸入generate_edge_embeddings得到要修改的edge，並直接修改成新的graph，用valid_modified_graphs儲存
        for emb, data, graph_emb in zip(valid_original_embeddings, self.valid_dataset, self.valid_graph_embedding):
            modify_edge = self.generate_edge_embeddings(data, emb, graph_emb) #pyg data是為了知道邊
            G = to_networkx(data, to_undirected=True)
            for decision in modify_edge:
                edge = decision[1]
                if (G.has_edge(edge[0], edge[1])):
                    G.remove_edge(edge[0], edge[1])
                else:
                    G.add_edge(edge[0], edge[1])
            self.valid_modified_graphs.append(G)
        
        valid_label_presever = 0
        for val_mod_graph, ori_opt in zip(self.valid_modified_graphs, valid_opt):
            mod_mvc = len(self.calculate_MVC(val_mod_graph))
            if ori_opt == mod_mvc:
                valid_label_presever += 1
        print(f"validation label presreved: {valid_label_presever}")
        
    def return_graph_embedding(self,G):
        """average all node embeddings to get graph embedding"""
        adj_matrix = nx.adjacency_matrix(G)
        adj_matrix = coo_matrix(adj_matrix)
        row = torch.from_numpy(adj_matrix.row.astype(np.int64))
        col = torch.from_numpy(adj_matrix.col.astype(np.int64))
        edge_index = torch.stack([row, col], dim=0)
        vec = Node2Vec(G, dimensions=self.Node2Vec_feature, walk_length=10, num_walks=10, workers=4, quiet=True)
        InitNodeEmb = vec.fit(window=5, min_count=1, batch_words=4)
        # embeddings = InitNodeEmb.wv # 50 node * 50 features
        node2vec_embeddings = [InitNodeEmb.wv[str(i)] for i in range(len(G.nodes))]
        node2vec_features_np = np.array(node2vec_embeddings)
        degrees = nx.degree(G)
        degree_values = [deg for node, deg in degrees]
        degree_values = torch.tensor(degree_values, dtype=torch.float)
        degree_normalized = (degree_values - degree_values.min()) / (degree_values.max() - degree_values.min())
        degree_features = degree_normalized.view(-1, 1)
        node2vec_features = torch.tensor(node2vec_features_np)
        combined_features = torch.cat([degree_features, node2vec_features], dim=1)
        data = Data(x=combined_features, edge_index=edge_index)
        embedding = self.gat(data)
        graph_embedding = self.get_graph_embedding([embedding])
        return graph_embedding
    

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
edge_sample_number = 50
learning_rate = 0.0001
mymodel = Modify_edge(num_features=128, graph_embedding_size=256, epoch=100, lr=learning_rate, modified_edge=edge_sample_number, GraphNumber=50, Graphsize=50, num_heads=4)
# mymodel.load_state_dict(torch.load("/workspace/backup/Modify_edge_model_1.pth"))
mymodel = mymodel.to(device)
print("mymodel")
for name, param in mymodel.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")
BCEloss_fn = nn.BCELoss()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mymodel.parameters(), lr=learning_rate,weight_decay=1e-5)

mymodel
gat.gat1.att_src: requires_grad=True
gat.gat1.att_dst: requires_grad=True
gat.gat1.bias: requires_grad=True
gat.gat1.lin_src.weight: requires_grad=True
gat.gat2.att_src: requires_grad=True
gat.gat2.att_dst: requires_grad=True
gat.gat2.bias: requires_grad=True
gat.gat2.lin_src.weight: requires_grad=True
mlp.fc1.weight: requires_grad=True
mlp.fc1.bias: requires_grad=True
mlp.fc2.weight: requires_grad=True
mlp.fc2.bias: requires_grad=True
mlp.fc3.weight: requires_grad=True
mlp.fc3.bias: requires_grad=True
mlp.fc4.weight: requires_grad=True
mlp.fc4.bias: requires_grad=True
mlp.fc5.weight: requires_grad=True
mlp.fc5.bias: requires_grad=True
classifier.fc1.weight: requires_grad=True
classifier.fc1.bias: requires_grad=True
classifier.fc2.weight: requires_grad=True
classifier.fc2.bias: requires_grad=True
classifier.fc3.weight: requires_grad=True
classifier.fc3.bias: requires_grad=True
classifier.fc4.weight: requires_grad=True
classifier.fc4.bias: requires_grad=True


In [5]:
diff_weight = 1 # MVC修改前後的差異
similarity_weight = 1 # 修改前後的圖的相似度
preserve_weight = 1 # 保留原本的label
classification_weight = 1 # 用來判定graph的result是否有相同的MVC
mlp_loss_weight = 0.1
for epoch in range(100):
    mymodel.train()
    similarity_loss , preserve_predict, labels_tensor, difference_loss, label_preserve, mlp_label, mlp_target = mymodel()
        
    mlp_loss = BCEloss_fn(mlp_target, mlp_label)   
    
    classifier_loss = criterion(preserve_predict, labels_tensor.float())
    
    none_preserve_loss = (mymodel.GraphNumber-label_preserve) /  mymodel.GraphNumber
    
    loss = classifier_loss * classification_weight + similarity_loss * similarity_weight + difference_loss * diff_weight + none_preserve_loss * preserve_weight + mlp_loss * mlp_loss_weight
    
    # loss = similarity_loss * similarity_weight + difference_loss * diff_weight + none_preserve * preserve_weight
      
    optimizer.zero_grad()
    loss.backward()
    # for name, param in mymodel.named_parameters():
    #     if param.grad is not None:
    #         print(f"Gradient of {name}: {param.grad.norm()}")
    #     else:
    #         print(f"Gradient of {name}: None")
    optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss.item()}, similarity_loss: {similarity_loss.item()}, difference_loss: {difference_loss}, none_label_preserve: {none_preserve_loss}, classifier_loss: {classifier_loss.item()}, mlp_loss : {mlp_loss.item()}")
    # if epoch % 2 == 0:
    #     print("validation")
    #     mymodel.eval()
    #     with torch.no_grad():
    #         mymodel.validation()

mod_mvc: 38, ori_mvc: 45, diff: 7
mod_mvc: 41, ori_mvc: 44, diff: 3
mod_mvc: 40, ori_mvc: 45, diff: 5
mod_mvc: 41, ori_mvc: 44, diff: 3
mod_mvc: 45, ori_mvc: 38, diff: 7
mod_mvc: 37, ori_mvc: 45, diff: 8
mod_mvc: 43, ori_mvc: 41, diff: 2
mod_mvc: 45, ori_mvc: 40, diff: 5
mod_mvc: 42, ori_mvc: 42, diff: 0
mod_mvc: 45, ori_mvc: 40, diff: 5
mod_mvc: 40, ori_mvc: 44, diff: 4
mod_mvc: 41, ori_mvc: 44, diff: 3
mod_mvc: 44, ori_mvc: 41, diff: 3
mod_mvc: 41, ori_mvc: 43, diff: 2
mod_mvc: 43, ori_mvc: 42, diff: 1
mod_mvc: 42, ori_mvc: 44, diff: 2
mod_mvc: 40, ori_mvc: 44, diff: 4
mod_mvc: 44, ori_mvc: 40, diff: 4
mod_mvc: 43, ori_mvc: 41, diff: 2
mod_mvc: 40, ori_mvc: 45, diff: 5
mod_mvc: 45, ori_mvc: 39, diff: 6
mod_mvc: 39, ori_mvc: 44, diff: 5
mod_mvc: 42, ori_mvc: 44, diff: 2
mod_mvc: 44, ori_mvc: 40, diff: 4
mod_mvc: 44, ori_mvc: 40, diff: 4
mod_mvc: 45, ori_mvc: 40, diff: 5
mod_mvc: 38, ori_mvc: 45, diff: 7
mod_mvc: 38, ori_mvc: 45, diff: 7
mod_mvc: 45, ori_mvc: 39, diff: 6
mod_mvc: 41, o